In [1]:
%run Common.ipynb

C:\Source\CSIRO\HealthyHabitatAI\.env
healthyhabitatanimals
healthyhabitatparragrass
ghNvkPl4DYLUUJh55y4pSg3BVV8TtPcHcX96P/zaDPXr9uuREdkGSCaao59d6xfZOfg4l9um1qfBXuNM9VGTTg==
jpEJGrFNi6PNLGOOD62rFBxjHGKTR4+DcppQNQQXi3P/GFmINKfqf8EZJoo73doIAnntv03ZSUsAC442IRR87Q==
processed
processed
f8fc2cd4db6b45a59d1ddd69a8685ddd
4610573e-864f-4f53-9f6a-c61ce520ed5a
5aeb092f-e91d-400f-bd88-e9516b108f20


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from PIL import Image, ImageFilter

In [3]:
%matplotlib inline

### Blob Storage

#### Para Grass Containers - *Site-Season*

In [4]:
para_grass_containers = blob_service_list_containers(healthy_habitat_paragrass_storage_name, healthy_habitat_paragrass_storage_key)

for container_name in para_grass_containers:
    print(container_name)

cannon-hill-bangkerreng
cannon-hill-kunumeleng
cannon-hill-wurrkeng
jabiru-dreaming-bangkerreng
jabiru-dreaming-kunumeleng
jabiru-dreaming-wurrkeng
resized
ubir-bangkerreng
ubir-kunumeleng
ubir-wurrkeng


### Resize

In [5]:
for container_name in para_grass_containers:
    blobs = blob_service_list_blobs(healthy_habitat_paragrass_storage_name, healthy_habitat_paragrass_storage_key, container_name)

    folders = set([blob.split('/')[0] for blob in blobs])

    for folder in folders:
        interim_folder_path = os.path.join(Path(os.getcwd()).parent, 'data', 'interim', container_name, folder)
        print(interim_folder_path)
        if not os.path.exists(interim_folder_path):
            os.makedirs(interim_folder_path)

    for blob_name in blobs:
        folder = blob_name.split('/')[0]
        file = blob_name.split('/')[1]

        # https://docs.microsoft.com/en-us/rest/api/storageservices/understanding-the-table-service-data-model#characters-disallowed-in-key-fields
        row_key = '{0}-{1}'.format(folder, file)

        filter = "PartitionKey eq '{0}' and RowKey eq '{1}'".format(container_name, row_key)

        entities = table_service_query_entities(healthy_habitat_paragrass_storage_name,
                                                healthy_habitat_paragrass_storage_key,
                                                healthy_habitat_paragrass_processed_table_name,
                                                filter)

        count = len([entity for entity in entities])

        if count == 0:
            print('Processing {0}'.format(blob_name))
            file_path = os.path.join(interim_folder_path, file)

            if not os.path.exists(file_path):
                blob_service_get_blob_to_path(healthy_habitat_paragrass_storage_name,
                                              healthy_habitat_paragrass_storage_key,
                                              container_name,
                                              blob_name,
                                              file_path)

            edge = 1144

            image = Image.open(file_path)
            image = image.crop((0, 0, image.size[1], image.size[1])) #left, upper, right, lower
            image = image.resize((edge, edge), resample=Image.LANCZOS)

            buffer = io.BytesIO()
            image.save(buffer, format='JPEG')

            path = '{0}/{1}/{2}'.format(container_name, folder, file)

            blob_service_create_blob_from_bytes(healthy_habitat_paragrass_storage_name,
                                                healthy_habitat_paragrass_storage_key,
                                                'resized',
                                                path,
                                                buffer.getvalue())

            entity = {'PartitionKey': container_name, 'RowKey': row_key, 'SemanticSegmentation': True}

            table_service_insert(healthy_habitat_paragrass_storage_name,
                                 healthy_habitat_paragrass_storage_key,
                                 healthy_habitat_paragrass_processed_table_name,
                                 entity)

            os.remove(file_path)

C:\Source\CSIRO\HealthyHabitatAI\data\interim\ubir-kunumeleng\2018-12-05-1200
Processing 2018-12-05-1200/DJI_0002.JPG
Processing 2018-12-05-1200/DJI_0003.JPG
Processing 2018-12-05-1200/DJI_0004.JPG
Processing 2018-12-05-1200/DJI_0005.JPG
Processing 2018-12-05-1200/DJI_0006.JPG
Processing 2018-12-05-1200/DJI_0007.JPG
Processing 2018-12-05-1200/DJI_0008.JPG
Processing 2018-12-05-1200/DJI_0009.JPG
Processing 2018-12-05-1200/DJI_0010.JPG
Processing 2018-12-05-1200/DJI_0011.JPG
Processing 2018-12-05-1200/DJI_0012.JPG
Processing 2018-12-05-1200/DJI_0013.JPG
Processing 2018-12-05-1200/DJI_0014.JPG
Processing 2018-12-05-1200/DJI_0015.JPG
Processing 2018-12-05-1200/DJI_0016.JPG
Processing 2018-12-05-1200/DJI_0017.JPG
Processing 2018-12-05-1200/DJI_0018.JPG
Processing 2018-12-05-1200/DJI_0019.JPG
Processing 2018-12-05-1200/DJI_0020.JPG
Processing 2018-12-05-1200/DJI_0021.JPG
Processing 2018-12-05-1200/DJI_0022.JPG
Processing 2018-12-05-1200/DJI_0023.JPG
Processing 2018-12-05-1200/DJI_0024.JPG
Pr

Processing 2018-12-05-1200/DJI_0205.JPG
Processing 2018-12-05-1200/DJI_0206.JPG
Processing 2018-12-05-1200/DJI_0207.JPG
Processing 2018-12-05-1200/DJI_0208.JPG
Processing 2018-12-05-1200/DJI_0209.JPG
Processing 2018-12-05-1200/DJI_0210.JPG
Processing 2018-12-05-1200/DJI_0211.JPG
Processing 2018-12-05-1200/DJI_0212.JPG
Processing 2018-12-05-1200/DJI_0213.JPG
Processing 2018-12-05-1200/DJI_0214.JPG
Processing 2018-12-05-1200/DJI_0215.JPG
Processing 2018-12-05-1200/DJI_0216.JPG
Processing 2018-12-05-1200/DJI_0217.JPG
Processing 2018-12-05-1200/DJI_0218.JPG
Processing 2018-12-05-1200/DJI_0219.JPG
Processing 2018-12-05-1200/DJI_0220.JPG
Processing 2018-12-05-1200/DJI_0221.JPG
Processing 2018-12-05-1200/DJI_0222.JPG
Processing 2018-12-05-1200/DJI_0223.JPG
Processing 2018-12-05-1200/DJI_0224.JPG
Processing 2018-12-05-1200/DJI_0225.JPG
Processing 2018-12-05-1200/DJI_0226.JPG
Processing 2018-12-05-1200/DJI_0227.JPG
Processing 2018-12-05-1200/DJI_0228.JPG
Processing 2018-12-05-1200/DJI_0229.JPG


Processing 2018-12-05-1200/DJI_0412.JPG
Processing 2018-12-05-1200/DJI_0413.JPG
Processing 2018-12-05-1200/DJI_0414.JPG
Processing 2018-12-05-1200/DJI_0415.JPG
Processing 2018-12-05-1200/DJI_0416.JPG
Processing 2018-12-05-1200/DJI_0417.JPG
Processing 2018-12-05-1200/DJI_0418.JPG
Processing 2018-12-05-1200/DJI_0419.JPG
Processing 2018-12-05-1200/DJI_0420.JPG
Processing 2018-12-05-1200/DJI_0421.JPG
Processing 2018-12-05-1200/DJI_0422.JPG
Processing 2018-12-05-1200/DJI_0423.JPG
Processing 2018-12-05-1200/DJI_0424.JPG
Processing 2018-12-05-1200/DJI_0425.JPG
Processing 2018-12-05-1200/DJI_0426.JPG
Processing 2018-12-05-1200/DJI_0427.JPG
Processing 2018-12-05-1200/DJI_0428.JPG
Processing 2018-12-05-1200/DJI_0429.JPG
Processing 2018-12-05-1200/DJI_0430.JPG
Processing 2018-12-05-1200/DJI_0431.JPG
Processing 2018-12-05-1200/DJI_0432.JPG
Processing 2018-12-05-1200/DJI_0433.JPG
Processing 2018-12-05-1200/DJI_0434.JPG
Processing 2018-12-05-1200/DJI_0435.JPG
Processing 2018-12-05-1200/DJI_0436.JPG


Processing 2018-12-05-1200/DJI_0619.JPG
Processing 2018-12-05-1200/DJI_0620.JPG
Processing 2018-12-05-1200/DJI_0621.JPG
Processing 2018-12-05-1200/DJI_0622.JPG
Processing 2018-12-05-1200/DJI_0623.JPG
Processing 2018-12-05-1200/DJI_0624.JPG
Processing 2018-12-05-1200/DJI_0625.JPG
Processing 2018-12-05-1200/DJI_0626.JPG
Processing 2018-12-05-1200/DJI_0627.JPG
Processing 2018-12-05-1200/DJI_0628.JPG
Processing 2018-12-05-1200/DJI_0629.JPG
Processing 2018-12-05-1200/DJI_0630.JPG
Processing 2018-12-05-1200/DJI_0631.JPG
Processing 2018-12-05-1200/DJI_0632.JPG
Processing 2018-12-05-1200/DJI_0633.JPG
Processing 2018-12-05-1200/DJI_0634.JPG
Processing 2018-12-05-1200/DJI_0635.JPG
Processing 2018-12-05-1200/DJI_0636.JPG
Processing 2018-12-05-1200/DJI_0637.JPG
Processing 2018-12-05-1200/DJI_0638.JPG
Processing 2018-12-05-1200/DJI_0639.JPG
Processing 2018-12-05-1200/DJI_0640.JPG
Processing 2018-12-05-1200/DJI_0641.JPG
Processing 2018-12-05-1200/DJI_0642.JPG
Processing 2018-12-05-1200/DJI_0643.JPG


In [ ]:
tagged_image_count = trainer.get_tagged_image_count(custom_vision_para_grass_project_id)
print('Tagged image count:', tagged_image_count)

untagged_image_count = trainer.get_untagged_image_count(custom_vision_para_grass_project_id)
print('Untagged image count:', untagged_image_count)

print('Image count', tagged_image_count + untagged_image_count)